In [2]:
pip install torch torchvision albumentations timm facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of facenet-pytorch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21

In [2]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import timm
from tqdm import tqdm

In [3]:
# --------------------------
# Config
# --------------------------
TRAIN_DIR = '/content/drive/MyDrive/FaceProjectData/Task_B/train'

BATCH_SIZE = 16  # smaller batch size for faster runs
EPOCHS = 5       # reduced epochs
LR = 1e-4
IMG_SIZE = 224
EMBEDDING_SIZE = 256  # reduced embedding size for speed
MAX_CLASSES = 200     # limit number of classes for quick test

In [4]:
# --------------------------
# Dataset
# --------------------------
class FaceTrainDataset(Dataset):
    def __init__(self, root_dir, transform=None, max_classes=None):
        self.samples = []
        self.transform = transform

        identities = sorted(os.listdir(root_dir))[:max_classes] if max_classes else os.listdir(root_dir)

        for identity in identities:
            id_path = os.path.join(root_dir, identity)
            if not os.path.isdir(id_path): continue

            good_img = os.path.join(id_path, f"{identity}.jpg")
            if os.path.exists(good_img):
                self.samples.append((good_img, identity))

            dist_dir = os.path.join(id_path, 'distortion')
            if os.path.exists(dist_dir):
                for fname in os.listdir(dist_dir):
                    if fname.endswith('.jpg') or fname.endswith('.png'):
                        self.samples.append((os.path.join(dist_dir, fname), identity))

        self.identity_to_label = {name: i for i, name in enumerate(sorted(set(x[1] for x in self.samples)))}
        self.samples = [(img, self.identity_to_label[label]) for img, label in self.samples]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [6]:
# --------------------------
# Model and ArcFace Loss
# --------------------------
class ArcFaceLoss(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.5):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)
        self.s = s
        self.m = m

    def forward(self, embeddings, labels):
        cosine = F.linear(F.normalize(embeddings), F.normalize(self.weight))
        theta = torch.acos(torch.clamp(cosine, -1.0 + 1e-7, 1.0 - 1e-7))
        target_logits = torch.cos(theta + self.m)
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, labels.view(-1,1), 1.0)
        output = cosine * (1 - one_hot) + target_logits * one_hot
        return output * self.s

class ArcFaceModel(nn.Module):
    def __init__(self, backbone_name='resnet18', embedding_size=256, num_classes=MAX_CLASSES):
        super().__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=True, num_classes=0)
        self.bn = nn.BatchNorm1d(self.backbone.num_features)
        self.fc = nn.Linear(self.backbone.num_features, embedding_size)
        self.arcface_head = ArcFaceLoss(embedding_size, num_classes)

    def forward(self, x, labels=None):
        x = self.backbone(x)
        x = self.bn(x)
        x = self.fc(x)
        if labels is not None:
            logits = self.arcface_head(x, labels)
            return logits, x
        return x

In [5]:
# --------------------------
# Train Loop
# --------------------------
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = T.Compose([
        T.Resize((IMG_SIZE, IMG_SIZE)),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize([0.5], [0.5])
    ])
    dataset = FaceTrainDataset(TRAIN_DIR, transform, max_classes=MAX_CLASSES)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

    model = ArcFaceModel(num_classes=len(dataset.identity_to_label)).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        model.train()
        for imgs, labels in tqdm(loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            imgs, labels = imgs.to(device), labels.to(device)
            logits, _ = model(imgs, labels)
            loss = criterion(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1} completed.")

    torch.save(model.state_dict(), 'arcface_finetuned_fast.pth')
    print("✅ Fast training done and model saved.")

In [8]:

if __name__ == "__main__":
    train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Epoch 1/5: 100%|██████████| 120/120 [14:08<00:00,  7.07s/it]


Epoch 1 completed.


Epoch 2/5: 100%|██████████| 120/120 [01:08<00:00,  1.75it/s]


...

Epoch 3 completed.


Epoch 4/5: 100%|██████████| 120/120 [01:08<00:00,  1.74it/s]


Epoch 4 completed.


Epoch 5/5: 100%|██████████| 120/120 [01:09<00:00,  1.72it/s]

Epoch 5 completed.
✅ Fast training done and model saved.


In [6]:
pip install scikit-learn

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset
from PIL import Image
import timm
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
# --------------------------
# Config
# --------------------------
VAL_DIR = '/content/drive/MyDrive/FaceProjectData/Task_B/val'
MODEL_PATH = 'arcface_finetuned_fast.pth'
IMG_SIZE = 224
EMBEDDING_SIZE = 256
MAX_CLASSES = 200  # same as training


In [12]:
# --------------------------
# Model and ArcFace Loss (same as training)
# --------------------------
class ArcFaceLoss(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.5):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)
        self.s = s
        self.m = m

    def forward(self, embeddings, labels):
        cosine = F.linear(F.normalize(embeddings), F.normalize(self.weight))
        theta = torch.acos(torch.clamp(cosine, -1.0 + 1e-7, 1.0 - 1e-7))
        target_logits = torch.cos(theta + self.m)
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, labels.view(-1,1), 1.0)
        output = cosine * (1 - one_hot) + target_logits * one_hot
        return output * self.s

class ArcFaceModel(nn.Module):
    def __init__(self, backbone_name='resnet18', embedding_size=256, num_classes=MAX_CLASSES):
        super().__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=True, num_classes=0)
        self.bn = nn.BatchNorm1d(self.backbone.num_features)
        self.fc = nn.Linear(self.backbone.num_features, embedding_size)
        self.arcface_head = ArcFaceLoss(embedding_size, num_classes)

    def forward(self, x, labels=None):
        x = self.backbone(x)
        x = self.bn(x)
        x = self.fc(x)
        if labels is not None:
            logits = self.arcface_head(x, labels)
            return logits, x
        return x

In [13]:
# --------------------------
# Validation Pipeline
# --------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ArcFaceModel(num_classes=MAX_CLASSES, embedding_size=EMBEDDING_SIZE).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize([0.5], [0.5])
])

def extract_embedding(img_path):
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model(image)[0].cpu().numpy()
    return embedding

In [14]:
# --------------------------
# Extract Gallery Embeddings
# --------------------------
gallery_embeddings = []
gallery_labels = []

for identity in sorted(os.listdir(VAL_DIR))[:MAX_CLASSES]:
    folder = os.path.join(VAL_DIR, identity)
    if not os.path.isdir(folder): continue

    good_img = os.path.join(folder, f"{identity}.jpg")
    if os.path.exists(good_img):
        emb = extract_embedding(good_img)
        gallery_embeddings.append(emb)
        gallery_labels.append(identity)

gallery_embeddings = np.vstack(gallery_embeddings)


In [ ]:
# --------------------------
# Match Distorted Images
# --------------------------
correct = 0
total = 0
for identity in sorted(os.listdir(VAL_DIR))[:MAX_CLASSES]:
    distortion_folder = os.path.join(VAL_DIR, identity, 'distortion')
    if not os.path.exists(distortion_folder): continue

    for fname in os.listdir(distortion_folder):
        img_path = os.path.join(distortion_folder, fname)
        query_emb = extract_embedding(img_path)

        sims = cosine_similarity(query_emb.reshape(1, -1), gallery_embeddings)[0]

        pred_idx = np.argmax(sims)
        predicted_identity = gallery_labels[pred_idx]

        if predicted_identity == identity:
            correct += 1
        total += 1

print(f"✅ Validation Accuracy: {correct}/{total} = {correct/total*100:.2f}%")